In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
def main():
    # Generate sample data (replace with your actual data)
    np.random.seed(42)
    n_samples = 1000
    n_features = 10
    X = np.random.randn(n_samples, n_features)
    y = np.random.randn(n_samples)

    # Scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Model parameters
    hidden_sizes = [32, 64, 128]
    pooling_types = ['max', 'avg']
    epochs_list = [5, 50, 100, 250, 350]
    optimizers = ['sgd', 'rmsprop', 'adam']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    sequence_length = 10

    # Create datasets
    train_dataset = BankDataset(X_train, y_train, sequence_length)
    val_dataset = BankDataset(X_test, y_test, sequence_length)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)

    results = []
    for hidden_size in hidden_sizes:
        for pooling_type in pooling_types:
            for num_epochs in epochs_list:
                for opt_name in optimizers:
                    print(f"\nTraining with parameters:")
                    print(f"Hidden Size: {hidden_size}, Pooling: {pooling_type}")
                    print(f"Epochs: {num_epochs}, Optimizer: {opt_name}")

                    model = RNNModel(n_features, hidden_size, 1, pooling_type)

                    if opt_name == 'sgd':
                        optimizer = optim.SGD(model.parameters(), lr=0.01)
                    elif opt_name == 'rmsprop':
                        optimizer = optim.RMSprop(model.parameters(), lr=0.001)
                    else:
                        optimizer = optim.Adam(model.parameters(), lr=0.001)

                    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                        optimizer, mode='min', factor=0.1, patience=3
                    )

                    criterion = nn.MSELoss()

                    train_losses, val_losses = train_model(
                        model, train_loader, val_loader,
                        criterion, optimizer, scheduler, num_epochs
                    )

                    # Plot training history
                    params = {
                        'hidden_size': hidden_size,
                        'pooling_type': pooling_type,
                        'epochs': num_epochs,
                        'optimizer': opt_name
                    }
                    plot_training_history(train_losses, val_losses, params)

                    results.append({
                        'hidden_size': hidden_size,
                        'pooling_type': pooling_type,
                        'epochs': num_epochs,
                        'optimizer': opt_name,
                        'final_train_loss': train_losses[-1],
                        'final_val_loss': val_losses[-1]
                    })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Analyze and visualize results
    analyze_results(results_df)

    # Create heatmaps for parameter combinations
    param_pairs = [
        ('hidden_size', 'epochs'),
        ('hidden_size', 'optimizer'),
        ('pooling_type', 'optimizer'),
        ('epochs', 'optimizer')
    ]

    for param1, param2 in param_pairs:
        plot_results_heatmap(results_df, param1, param2)

    # Save results
    results_df.to_csv('bank.csv', index=False)
    print("\nResults saved to 'bank.csv'")

In [14]:
# [Previous Dataset and Model classes remain the same as in the first artifact]
class BankDataset(Dataset):
    def __init__(self, X, y, sequence_length):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.X) - self.sequence_length

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.sequence_length],
                self.y[idx+self.sequence_length])

In [15]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, pooling_type='max'):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.rnn(x)
        if self.pooling_type == 'max':
            pooled = torch.max(output, dim=1)[0]
        else:
            pooled = torch.mean(output, dim=1)
        out = self.fc(pooled)
        return out

In [16]:
def plot_training_history(train_losses, val_losses, params):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title(f'Training History\nParams: {params}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_results_heatmap(results_df, x_param, y_param, value='final_val_loss'):
    pivot_table = results_df.pivot_table(
        values=value,
        index=y_param,
        columns=x_param,
        aggfunc='mean'
    )

    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_table, annot=True, fmt='.4f', cmap='viridis')
    plt.title(f'{value} comparison: {y_param} vs {x_param}')
    plt.show()

def analyze_results(results_df):
    # Best configurations
    best_config = results_df.loc[results_df['final_val_loss'].idxmin()]
    print("\nBest Configuration:")
    for param, value in best_config.items():
        print(f"{param}: {value}")

    # Parameter analysis
    params = ['hidden_size', 'pooling_type', 'epochs', 'optimizer']
    for param in params:
        print(f"\nAnalysis by {param}:")
        analysis = results_df.groupby(param)['final_val_loss'].agg(['mean', 'std', 'min', 'max'])
        print(analysis)

        plt.figure(figsize=(10, 6))
        sns.boxplot(x=param, y='final_val_loss', data=results_df)
        plt.title(f'Loss Distribution by {param}')
        plt.xticks(rotation=45)
        plt.show()